# Quantum SVM with Quantum Annealing

In this tutorial, a quantum version of the Support Vector Machine (SVM) algorithm based on Quantum Annealing (QA) is presented.

## Introduction

Before starting, some basic theoretical concepts are introduced.

### Support Vector Machine (SVM)

Given a training set $T=\left\{\left(\textbf{x}_{n}, y_{n}\right): n=0, \ldots, N-1\right\}$, where $\textbf{x}_{n} \in \mathbb{R}^{d}$ are the feature vectors and $y_n \in \{-1,1\}$ are the binary labels, the conventional dual formulation of a SVM can be defined as the following quadratic programming problem: 
$$
\min_{\pmb{\alpha}}L(\pmb{\alpha})=\frac{1}{2} \sum_{n m} \alpha_{n} \alpha_{m} y_{n} y_{m} k\left(\mathbf{x}_{n}, \mathbf{x}_{m}\right)-\sum_{n} \alpha_{n},
$$
subject to the constraints
$$
\begin{split}
\quad 0 \leq \alpha_{n} \leq C  \> \> \text { ; } \> \> \sum_{n} \alpha_{n} y_{n}=0,
\end{split}
$$
where $\pmb{\alpha}=\{\alpha_n:n=0,...,N-1\}$ are the variables of the optimization problem (the Lagrange multipliers), $k(\mathbf{x}_n, \mathbf{x}_m)$ is the kernel function and $C$ is the regularization parameter.


### Quadratic Unconstrained Binary Optimization (QUBO)

A QUBO problem is an optimization problem with the following properties:
* _Quadratic_: the energy function $E$ (also called cost function) is a quadratic polynomial;
* _Unconstrained_: no constraints are required;
* _Binary_: the unknowns are a set of $n$ binary variables $x \in \{ 0,1 \}^n$.

It can be represented by a weighted graph $G$ where the vertices $V(G)$ are the variables and the edges $E(G)$ are the weights associated to each variables product. 

<p align="center">
  <img src="https://drive.google.com/uc?export=view&id=1l9cZmaN7DFrQAsqkdmhuNQkC2qrlWRqx"
  width="30%"/>
</p>

The mathematical formulation of the QUBO problem is:

$$
\min_{x \in \{0,1 \}^n} \sum_{(ij) \in E(G)} Q_{ij}x_i x_j + \sum_{i \in V(G)}Q_{ii}x_i = \min_{x \in \{0,1 \}^n}  x^\top Q x
$$

where the graph $G$ is defined by the QUBO matrix $Q$ of dimensions $n\times n$ (also called adjacency matrix). For example, the QUBO matrix associated with the graph above is:

$$
Q=\begin{bmatrix}
0 & 3 & 0 & 7 & 8\\
3 & 0 & 1 & 4 & 0\\
0 & 1 & 0 & 2 & 0\\
7 & 4 & 2 & 0 & 3\\
8 & 0 & 0 & 3 & 0\\
\end{bmatrix}
$$


### Minor Embedding
_D-Wave Advantage_ is a quantum computing machine based on quantum annealing, with more than 5000 qubits and more than 35000 couplers (connections between couples of qubits). Here is a representation of the Pegasus qubit architecture of Advantage.


<figure> 
  <p align="center">
    <img src="https://drive.google.com/uc?export=view&id=1uGqhnHDV54dPSG6Tp7oTiJMyQgkGKlzl" width="40%"/>
    <figcaption>
    <p align="center">
        Source: D-Wave Documentation
    </p>
    </figcaption>
  </p>
</figure>

A problem can be submitted to a D-Wave quantum annealer in the form of a QUBO problem. To do so, a process called _minor embedding_ is needed. The binary variables of the problem are mapped to physical qubits, with the requirement that linked binary variables correspond to linked physical qubits.

<figure> 
  <p align="center">
    <img src="https://drive.google.com/uc?export=view&id=1_2dEm0UD-IxDDL2rDjghqsdyEsT7OxcI" width="60%"/>
    <figcaption>
    <p align="center">
        Source: D-Wave Systems
    </p>
    </figcaption>
  </p>
</figure>


## Initial Steps


### Library Installation

Fortunately, in Colab we have access to a cloud environment where we can freely access computing machines and easily run Python codes without initial configuration. All we need to do is import the needed libraries. In our case, we will just have to manually install the libraries for implementing quantum algorithms and submitting tasks to D-Wave quantum annealers.

In [ ]:
try:
  import google.colab
  IN_COLAB = True
except:
  IN_COLAB = False

if IN_COLAB:
    !pip install dwave-ocean-sdk

### Account Configuration

A configuration file must be created in order to set up the solver. Launch the following interactive command and complete the fields as indicated in the image. The authentication token can be found by signing into your D-Wave account on https://cloud.dwavesys.com/leap/.

<p align="center">
  <img src="https://drive.google.com/uc?export=view&id=16ajHppOFefGtgdRDtAWlcYSSC8PbNUHo" width="60%"/>
</p>


In [ ]:
!dwave config create

Check the correctness of the configuration:

In [ ]:
!dwave ping

### Library Imports

We import and rename the libraries we will be using during the tutorial.

In [4]:
import os
import re
import gc
import json
import glob
import shutil
import pickle
import matplotlib.pyplot as plt
import numpy as np
import dimod
import numpy.lib.recfunctions as rfn
from dwave.system.samplers import DWaveSampler
from dwave.system.composites import EmbeddingComposite
from dwave.system.composites import LazyFixedEmbeddingComposite
from dimod import BinaryQuadraticModel

## Quantum SVM Algorithm

Now we are ready to introduce our Quantum SVM algorithm.

### Mathematical Formulation

The starting point of our algorithm is the dual formulation of the SVM (see **Introduction**). In order to leverage quantum annealing, we have to follow this golden rule:

> **The optimization problem must be reframed as a QUBO problem.**

Let's compare the classic SVM with the properties of QUBO:
* _Quadratic_: the energy function is a quadratic polynomial w.r.t. the unknowns $\alpha_{n}$  ✔️
* _Unconstrained_: two constraints are required ❌
* _Binary_: the unknowns $\alpha_{n}$ are real, non-negative values ❌

Let's start from the _binary_ requirement. A strategy is to discretize the $\alpha_n$ solution space and to use binary variables to encode each discretized value of $\alpha_{n}$ using the following encoding:

$$
\alpha_{n}=\sum_{k=0}^{K-1} B^{k-E} a_{K n+k},
$$

where $a_{K n+k} \in\{0,1\}$ are binary variables, $K$ is the number of binary variables used to encode $\alpha_{n}$, $B\geq 0$ is the base, and $E \geq 0$ is a parameter that allows for negative exponents.

This encoding already solves the first SVM constraint, since it bounds the possible values of $\alpha_n$.
Regarding the second constraint, we can directly add it into the energy function $E$ and a squared penalty term, multiplied by $\xi/2$.

With these assumptions, the energy function can be written as
$$
E=\sum_{n, m=0}^{N-1} \sum_{k, j=0}^{K-1} a_{K n+k} \widetilde{Q}_{K n+k, K m+j} a_{K m+j},
$$

where $\widetilde{Q}$ is the QUBO matrix of size $K N \times K N$ given by 

$$
\widetilde{Q}_{K n+k, K m+j} =\frac{1}{2} B^{k+j-2E} y_{n} y_{m}\left(k\left(\mathbf{x}_{n}, \mathbf{x}_{m}\right)+\xi\right) -\delta_{n m} \delta_{k j} B^{k-E}.
$$

### Functions Definition

For the sake of clarity and conciseness, we split the algorithm in functions, each one performing a specific task.

* The function ``decode`` converts a vector of binary values to a vector real values $\alpha_n$.



In [13]:
def decode(binary, B=2, K=3, E=0):
    N = len(binary) // K  # Number of real variables we have to decode (alphas)
    Bvec = float(B) ** (np.arange(K)-E)
    return np.fromiter(binary,float).reshape(N,K) @ Bvec

* The function ``kernel`` implements the kernel function. The Gaussian kernel is selected:
$$
k\left(\mathbf{x}_{n}, \mathbf{x}_{m}\right) = e^{-\gamma ||\mathbf{x}_{n}- \mathbf{x}_{m}||^2}
$$

``xn`` has shape ``(N,D)`` and ``xm`` has shape ``(M,D)``. The result is a kernel matrix of shape ``(N,M)``. If ``gamma=-1``, a linear kernel is selected.

In [5]:
def kernel(xn, xm, gamma=-1):
    if gamma == -1:
        return xn @ xm.T
    xn = np.atleast_2d(xn)
    xm = np.atleast_2d(xm)
    return np.exp(-gamma * np.sum((xn[:,None] - xm[None,:])**2, axis=-1))

* The function ``gen_svm_qubos`` generates the QUBO matrix $Q$ according to its definition.



In [6]:
def gen_svm_qubos(data,label,B,K,xi,gamma,E,path):

    N = len(data)
    
    if not os.path.isfile(path+'Q.npy'):
      Q = ## MISSING CODE HERE: Initialize QUBO matrix to zero
    
      print(f'Creating the QUBO of size {Q.shape}')
      for n in range(N):
          for m in range(N):
              for k in range(K):
                  for j in range(K):
                      Q[K*n+k,K*m+j] = ## MISSING CODE HERE: QUBO matrix
                      if n == m and k == j:
                          Q[K*n+k,K*m+j] += - B**(k-E)


      Q = np.triu(Q) + np.tril(Q,-1).T
    else:
      Q = np.load(path+'Q.npy')
    print(f'Extracting nodes and couplers')
    qubo_nodes = np.asarray([[n, n, Q[n,n]] for n in range(len(Q))])
    qubo_couplers = np.asarray([[n, m, Q[n,m]] for n in range(len(Q)) for m in range(n+1,len(Q)) if not np.isclose(Q[n,m],0)])
    qubo_couplers = qubo_couplers[np.argsort(-np.abs(qubo_couplers[:,2]))]


    print(f'Saving {len(qubo_nodes)} nodes and {len(qubo_couplers)} couplers for {path}')
    os.makedirs(path, exist_ok=True)
    np.save(path+'Q.npy', Q)
    np.savetxt(path+'qubo_nodes.dat', qubo_nodes, fmt='%g', delimiter='\t')
    np.savetxt(path+'qubo_couplers.dat', qubo_couplers, fmt='%g', delimiter='\t')
    
    return



The function ``dwave_run_embedding`` submits the QUBO problem to the selected quantum annealer.

In [7]:
def dwave_run_embedding(data,label,path_in,annealing_times,chain_strengths,em_id,solver='Advantage_system1.1'):
    
    MAXRESULTS = 20
    match = re.search('run_([^/]*)_B=(.*)_K=(.*)_xi=(.*)_E=(.*)_gamma=([^/]*)', path_in) 

    data_key = match.group(1)
    B = int(match.group(2))
    K = int(match.group(3))
    xi = float(match.group(4))
    gamma = float(match.group(6))
    E = int(match.group(5))

    path = path_in+ ('/' if path_in[-1] != '/' else '')
    qubo_couplers = np.loadtxt(path+'qubo_couplers.dat')
    qubo_nodes = np.loadtxt(path+'qubo_nodes.dat')
    qubo_nodes = np.array([[i,i,(qubo_nodes[qubo_nodes[:,0]==i,2][0] if i in qubo_nodes[:,0] else 0.)] for i in np.arange(np.concatenate((qubo_nodes,qubo_couplers))[:,[0,1]].max()+1)])  # to make sure every (i,i) occurs in the qubo in increasing order such that the variable order in BinaryQuadraticModel is consistent (see locate wrongenergies-* github issue)
    maxcouplers = len(qubo_couplers)

    if not 'train' in data_key:
        raise Exception(f'careful: datakey={data_key} => youre trying to train on a validation / test set!')

    couplerslist = [min(7500,maxcouplers)]
    for trycouplers in [5000, 2500, 2000, 1800, 1600, 1400, 1200, 1000, 500]:
        if maxcouplers > trycouplers:
            couplerslist += [trycouplers]


    sampler = LazyFixedEmbeddingComposite(DWaveSampler(solver=solver))
    for n in range(0,len(chain_strengths)):
        for m in range(0,len(annealing_times)):
            if n==0 and m==0:
                for couplers in couplerslist:
                    Q = { (q[0], q[1]): q[2] for q in np.vstack((qubo_nodes, qubo_couplers[:couplers])) }
                    maxstrength=np.max( np.abs( list( Q.values() ) ) )
                    pathsub = path + f'result_couplers={couplers}/'
                    os.makedirs(pathsub, exist_ok=True)
                    embedding_file_name=pathsub+f'embedding_id{em_id}'
                    if os.path.isfile(embedding_file_name):
                        embedding_file=open(embedding_file_name,'r')
                        embedding_data=eval(embedding_file.read())
                        embedding_file.close()
                        sampler._fix_embedding(embedding_data)
                    print(f'running {pathsub} with {len(qubo_nodes)} nodes and {couplers} couplers for embedding {em_id}')
    
                    ordering = np.array(list(BinaryQuadraticModel.from_qubo(Q)))
                    if not (ordering == np.arange(len(ordering),dtype=ordering.dtype)).all():
                        print(f'WARNING: variables are not correctly ordered! path={path} ordering={ordering}')

                    try:
                        print(f'Running chain strength {chain_strengths[n]} and annealing time {annealing_times[m]}\n')
                        response = sampler.sample_qubo(Q, num_reads=5000, annealing_time=annealing_times[0], chain_strength=chain_strengths[0]*maxstrength)
                        if not os.path.isfile(embedding_file_name):
                            embedding_file=open(embedding_file_name,'w')
                            print('Embedding found. Saving...')
                            embedding_file.write(repr(sampler.embedding))
                            embedding_file.close()
            
                    except ValueError as v:
                        print(f' -- no embedding found, removing {pathsub} and trying less couplers')
                        shutil.rmtree(pathsub)
                        sampler = LazyFixedEmbeddingComposite(DWaveSampler(solver=solver))
                        continue
                    break
            else:
                print(f'Running chain strength {chain_strengths[n]} and annealing time {annealing_times[m]}')
                response = sampler.sample_qubo(Q, num_reads=5000, annealing_time=annealing_times[m], chain_strength=chain_strengths[n]*maxstrength)

            pathsub_ext=pathsub+f'embedding{em_id}_rcs{chain_strengths[n]}_ta{annealing_times[m]}_'
            save_json(pathsub_ext+'info.json', response.info)

            samples = np.array([''.join(map(str,sample)) for sample in response.record['sample']]) 
            unique_samples, unique_idx, unique_counts = np.unique(samples, return_index=True, return_counts=True)
            unique_records = response.record[unique_idx]
            result = rfn.merge_arrays((unique_samples, unique_records['energy'], unique_counts, unique_records['chain_break_fraction'])) 
            result = result[np.argsort(result['f1'])]
            np.savetxt(pathsub_ext+'result.dat', result[:MAXRESULTS], fmt='%s', delimiter='\t', header='\t'.join(response.record.dtype.names), comments='')

            alphas = np.array([decode(sample,B,K,E) for sample in result['f0'][:MAXRESULTS]])
            np.save(pathsub_ext+f'alphas.npy', alphas)
            gc.collect()
    
    return pathsub


* Some additional functions:

In [9]:
def plot_qa_svm(alphas, data, label, gamma, B, K, xylim=[-0.7, 1.7], notitle=False, filled=False,
                plot_support=True):
    C = (B ** np.arange(K)).sum()
    b = eval_offset_avg(alphas, data, label, gamma, C)
    result = np.copysign(1, eval_classifier(data, alphas, data, label, gamma, b))

    xsample = np.arange(xylim[0], xylim[1] + .01, .05)
    x1grid, x2grid = np.meshgrid(xsample, xsample)
    X = np.vstack((x1grid.ravel(), x2grid.ravel())).T  # ...xD for kernel contraction
    FX = eval_classifier(X, alphas, data, label, gamma, b).reshape(len(xsample), len(xsample))

    plt.pcolor(x1grid, x2grid, FX, cmap='coolwarm')
    plt.contour(x1grid, x2grid, FX, [0.], linewidths=3, colors='black')

    if not filled:
        plt.scatter(data[result == 1][:, 0], data[result == 1][:, 1], c='r', marker=(8, 2, 0), linewidth=0.5)
        plt.scatter(data[result != 1][:, 0], data[result != 1][:, 1], c='b', marker='+', linewidth=1)
        plt.scatter(data[label == 1][:, 0], data[label == 1][:, 1], edgecolors='r', marker='s', linewidth=0.5,
                    facecolors='none')
        plt.scatter(data[label != 1][:, 0], data[label != 1][:, 1], edgecolors='b', marker='o', linewidth=1,
                    facecolors='none')
    else:
        plt.scatter(data[label == 1][:, 0], data[label == 1][:, 1], edgecolors='r', marker='s', linewidth=1,
                    facecolors='r')
        plt.scatter(data[label != 1][:, 0], data[label != 1][:, 1], edgecolors='b', marker='o', linewidth=1,
                    facecolors='b')

    # plot support vectors
    if plot_support:
        support_vectors = data[np.nonzero(alphas)[0]]
        plt.scatter(support_vectors[:, 0], support_vectors[:, 1], edgecolors='green', facecolors='none', s=300,
                    linewidth=1)

    if not notitle:
        support_vectors = data[np.nonzero(alphas)[0]]
        plt.title(
            str(len(support_vectors)) + ' SVs, ' + str(round(((result == label).sum() / len(label)), 2)) + ' accuracy')

    plt.xlim(*xylim)
    plt.ylim(*xylim)

def eval_offset_avg(alphas, data, label, gamma, C, useavgforb=True):
    cross = eval_classifier(data, alphas, data, label,
                            gamma)
    if useavgforb:
        return np.sum(alphas * (C - alphas) * (label - cross)) / np.sum(alphas * (C - alphas))
    else:
        if np.isclose(np.sum(alphas * (C - alphas)), 0):
            print('no support vectors found, discarding this classifer')
            return np.nan
        bcandidates = [np.sum(alphas * (C - alphas) * (label - cross)) / np.sum(
            alphas * (C - alphas))]
        crosssorted = np.sort(cross)
        crosscandidates = -(crosssorted[1:] + crosssorted[
                                              :-1]) / 2
        bcandidates += sorted(crosscandidates,
                              key=lambda x: abs(x - bcandidates[0]))
        bnumcorrect = [(label == np.sign(cross + b)).sum() for b in bcandidates]
        return bcandidates[np.argmax(bnumcorrect)]

def save_json(filename, var):
    with open(filename,'w') as f:
        f.write(str(json.dumps(var, indent=4, sort_keys=True, separators=(',', ': '), ensure_ascii=False)))

def eval_classifier(x, alphas, data, label, gamma,
                    b=0):
    return np.sum((alphas * label)[:, None] * kernel(data, x, gamma), axis=0) + b



### Dataset and Parameters Definition

We test our binary classification algorithm on a simple dataset consisting of $N=40$ samples.

In [ ]:
X_train = np.array([
       [ 0.0401,  0.1415],
       [-0.1211, -0.0228],
       [-0.1688,  0.2911],
       [ 0.2293,  0.7856],
       [-1.0831, -0.1049],
       [ 0.1626, -1.0458],
       [ 0.311 , -0.1032],
       [-0.9376, -0.2677],
       [ 0.7176, -0.7998],
       [-0.0197,  0.3323],
       [-0.1306,  0.1053],
       [ 0.0811, -0.1263],
       [ 0.3391,  0.1625],
       [ 0.6602,  0.0843],
       [-0.1588,  0.1835],
       [-0.3887,  0.2282],
       [ 0.2555,  0.0709],
       [ 0.1717, -0.9481],
       [-0.0288, -0.1503],
       [-0.7354,  0.5788],
       [-0.6483, -1.2539],
       [ 0.1448,  1.0718],
       [ 0.803 , -0.7706],
       [-0.1384, -0.9474],
       [-0.0025,  0.0779],
       [-0.686 ,  0.9516],
       [ 0.1013, -0.5189],
       [ 0.8678,  0.0948],
       [-0.0021, -0.082 ],
       [-1.0192, -0.3854],
       [ 0.1661, -0.1268],
       [ 0.2193,  0.0973],
       [-0.1468,  1.0086],
       [-0.0195,  0.503 ],
       [ 0.0475,  0.3345],
       [ 0.7212, -0.1479],
       [-0.5998,  0.5036],
       [-0.1889, -0.0499],
       [ 0.9087,  0.525 ],
       [ 0.5112, -0.4113]])
Y_train = np.array(
      [ 1.,  1.,  1., -1., -1., -1.,  1., -1., -1.,  1.,  1.,  1.,  1.,
       -1.,  1.,  1.,  1., -1.,  1., -1., -1., -1., -1., -1.,  1., -1.,
        1., -1.,  1., -1.,  1.,  1., -1.,  1.,  1., -1., -1.,  1., -1.,
       -1.])

plt.figure(0)
plt.title('Training set')
plt.xlim(-2, 2)
plt.ylim(-2, 2)
plot = plt.scatter(X_train[:, 0], X_train[:, 1], c=Y_train, s=50, cmap='autumn')

Now we define the parameters of the algorithm.

In [11]:
outputpath='output/run_train'
maxalphas=20

Bs=[2]
Ks=[2]
xis=[0]
gammas=[5]
Es=[0]
annealing_times=[10]
chain_strengths=[1]
embeddings=[0]

### Problem Submission

Now we can launch the algorithm, once for every parameter combination.

In [ ]:
for B in Bs:
    for K in Ks:
        for gamma in gammas:
            for xi in xis:
                for E in Es:
                    path=outputpath+f'_B={B}_K={K}_xi={xi}_E={E}_gamma={gamma}/'
                    gen_svm_qubos(X_train,Y_train,B,K,xi,gamma,E,path)
                    for i in embeddings:
                        dwave_run_embedding(X_train,Y_train,path,annealing_times,chain_strengths,i)


In [ ]:
for B in Bs:
    for K in Ks:
        for gamma in gammas:
            for xi in xis:
                for E in Es:
                    dirs=glob.glob(outputpath+f'_B={B}_K={K}_xi={xi}_E={E}_gamma={gamma}/result_couplers=*')
                    path=dirs[0]+'/'
                    for emb in embeddings:
                        for c in chain_strengths:
                            for t in annealing_times:
                              alphas=np.load(path+f'embedding{emb}_rcs{c}_ta{t}_alphas.npy')
                              plot_qa_svm(alphas[0], X_train, Y_train, gamma, B,K, [-2,2], False, True, True)
